<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Random-Forest-Balanced" data-toc-modified-id="Random-Forest-Balanced-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Random Forest Balanced</a></span></li><li><span><a href="#XGBoost" data-toc-modified-id="XGBoost-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>XGBoost</a></span></li><li><span><a href="#Adaboost" data-toc-modified-id="Adaboost-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Adaboost</a></span></li><li><span><a href="#CatBoost" data-toc-modified-id="CatBoost-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>CatBoost</a></span></li></ul></div>

In [7]:
import pandas as pd
import numpy as np
import seaborn as sns

sns.set()
%matplotlib inline

In [3]:
df = pd.read_csv(r'C:\Users\Tiago McDonald\Downloads\telecom_churn.csv')

In [4]:
df.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [9]:
df.describe()

,Account length,Area code,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls
count,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000
mean,101.064806,437.182418,8.099010,179.775098,100.435644,30.562307,200.980348,100.114311,17.083540,200.872037,100.107711,9.039325,10.237294,4.479448,2.764581,1.562856
std,39.822106,42.371290,13.688365,54.467389,20.069084,9.259435,50.713844,19.922625,4.310668,50.573847,19.568609,2.275873,2.791840,2.461214,0.753773,1.315491
min,1.000000,408.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,23.200000,33.000000,1.040000,0.000000,0.000000,0.000000,0.000000
25%,74.000000,408.000000,0.000000,143.700000,87.000000,24.430000,166.600000,87.000000,14.160000,167.000000,87.000000,7.520000,8.500000,3.000000,2.300000,1.000000
50%,101.000000,415.000000,0.000000,179.400000,101.000000,30.500000,201.400000,100.000000,17.120000,201.200000,100.000000,9.050000,10.300000,4.000000,2.780000,1.000000
75%,127.000000,510.000000,20.000000,216.400000,114.000000,36.790000,235.300000,114.000000,20.000000,235.300000,113.000000,10.590000,12.100000,6.000000,3.270000,2.000000
max,243.000000,510.000000,51.000000,350.800000,165.000000,59.640000,363.700000,170.000000,30.910000,395.000000,175.000000,17.770000,20.000000,20.000000,5.400000,9.000000


In [12]:
df.drop('State', axis=1, inplace=True)

In [14]:
df['International plan'] = df['International plan'].map({'Yes':True,'No':False})
df['Voice mail plan'] = df['Voice mail plan'].map({'Yes':True,'No':False})

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, f1_score, precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [15]:
!pip install xgboost
!pip install catboost

In [17]:
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [18]:
X_train, X_test,y_train,y_test = train_test_split(df.drop('Churn',axis=1), \
                                                  df.Churn, \
                                                  test_size=1/3, \
                                                  random_state=42, \
                                                  stratify=df.Churn #Keeps the same proportion of Churn on the train and test sets
                                                 )

In [19]:
model1=DecisionTreeClassifier()
model1.fit(X_train, y_train)
y_pred1=model1.predict(X_test)
conf1=confusion_matrix(y_test, y_pred1)
acc1=accuracy_score(y_test, y_pred1)
rec1=recall_score(y_test, y_pred1)
pre1=precision_score(y_test, y_pred1)
f11=f1_score(y_test, y_pred1)
display(conf1)
print('Accuracy', acc1)
print('Recall', rec1)
print('Precision', pre1)
print('F1', f11)

array([[906,  44],
       [ 54, 107]], dtype=int64)

Accuracy 0.9117911791179117
Recall 0.6645962732919255
Precision 0.7086092715231788
F1 0.6858974358974359


In this dataset, analyzing the Recall is the most important because we want to reduce the proportion of False Negatives (People who we predicted were not going to Churn but did)

In [20]:
# Random Forest
model1=RandomForestClassifier()
model1.fit(X_train, y_train)
y_pred1=model1.predict(X_test)
conf1=confusion_matrix(y_test, y_pred1)
acc1=accuracy_score(y_test, y_pred1)
rec1=recall_score(y_test, y_pred1)
pre1=precision_score(y_test, y_pred1)
f11=f1_score(y_test, y_pred1)
display(conf1)
print('Accuracy', acc1)
print('Recall', rec1)
print('Precision', pre1)
print('F1', f11)

array([[941,   9],
       [ 44, 117]], dtype=int64)

Accuracy 0.9522952295229523
Recall 0.7267080745341615
Precision 0.9285714285714286
F1 0.8153310104529616


## Random Forest Balanced

In [22]:
model1=RandomForestClassifier(class_weight='balanced')
model1.fit(X_train, y_train)
y_pred1=model1.predict(X_test)
conf1=confusion_matrix(y_test, y_pred1)
acc1=accuracy_score(y_test, y_pred1)
rec1=recall_score(y_test, y_pred1)
pre1=precision_score(y_test, y_pred1)
f11=f1_score(y_test, y_pred1)
display(conf1)
print('Accuracy', acc1)
print('Recall', rec1)
print('Precision', pre1)
print('F1', f11)

array([[939,  11],
       [ 47, 114]], dtype=int64)

Accuracy 0.9477947794779478
Recall 0.7080745341614907
Precision 0.912
F1 0.7972027972027971


## XGBoost

In [23]:
model1=XGBClassifier()
model1.fit(X_train, y_train)
y_pred1=model1.predict(X_test)
conf1=confusion_matrix(y_test, y_pred1)
acc1=accuracy_score(y_test, y_pred1)
rec1=recall_score(y_test, y_pred1)
pre1=precision_score(y_test, y_pred1)
f11=f1_score(y_test, y_pred1)
display(conf1)
print('Accuracy', acc1)
print('Recall', rec1)
print('Precision', pre1)
print('F1', f11)

array([[932,  18],
       [ 44, 117]], dtype=int64)

Accuracy 0.9441944194419442
Recall 0.7267080745341615
Precision 0.8666666666666667
F1 0.7905405405405406


## Adaboost

In [29]:
model1=AdaBoostClassifier()
model1.fit(X_train, y_train)
y_pred1=model1.predict(X_test)
conf1=confusion_matrix(y_test, y_pred1)
acc1=accuracy_score(y_test, y_pred1)
rec1=recall_score(y_test, y_pred1)
pre1=precision_score(y_test, y_pred1)
f11=f1_score(y_test, y_pred1)
display(conf1)
print('Accuracy', acc1)
print('Recall', rec1)
print('Precision', pre1)
print('F1', f11)

array([[906,  44],
       [102,  59]], dtype=int64)

Accuracy 0.8685868586858686
Recall 0.36645962732919257
Precision 0.5728155339805825
F1 0.446969696969697


## CatBoost

In [31]:
model1=CatBoostClassifier()
model1.fit(X_train, y_train)
y_pred1=model1.predict(X_test)
conf1=confusion_matrix(y_test, [eval(i) for i in y_pred1])#Because Catboost converts the booleans to Text. Eval function converts back to boolean
acc1=accuracy_score(y_test, [eval(i) for i in y_pred1])
rec1=recall_score(y_test, [eval(i) for i in y_pred1])
pre1=precision_score(y_test, [eval(i) for i in y_pred1])
f11=f1_score(y_test, [eval(i) for i in y_pred1])
display(conf1)
print('Accuracy', acc1)
print('Recall', rec1)
print('Precision', pre1)
print('F1', f11)

Learning rate set to 0.014488
0:	learn: 0.6810795	total: 5.78ms	remaining: 5.77s
1:	learn: 0.6671237	total: 9.62ms	remaining: 4.8s
2:	learn: 0.6547928	total: 13.6ms	remaining: 4.52s
3:	learn: 0.6388798	total: 17ms	remaining: 4.24s
4:	learn: 0.6255185	total: 20.5ms	remaining: 4.08s
5:	learn: 0.6107657	total: 24.1ms	remaining: 3.99s
6:	learn: 0.5977423	total: 27.9ms	remaining: 3.95s
7:	learn: 0.5868114	total: 31.5ms	remaining: 3.91s
8:	learn: 0.5773122	total: 36ms	remaining: 3.97s
9:	learn: 0.5651216	total: 40.3ms	remaining: 3.98s
10:	learn: 0.5549107	total: 43.9ms	remaining: 3.95s
11:	learn: 0.5451468	total: 47.6ms	remaining: 3.92s
12:	learn: 0.5351335	total: 50.8ms	remaining: 3.86s
13:	learn: 0.5260024	total: 54.1ms	remaining: 3.81s
14:	learn: 0.5170633	total: 57ms	remaining: 3.75s
15:	learn: 0.5050288	total: 60ms	remaining: 3.69s
16:	learn: 0.4965513	total: 63.1ms	remaining: 3.65s
17:	learn: 0.4896704	total: 66.2ms	remaining: 3.61s
18:	learn: 0.4817171	total: 69.2ms	remaining: 3.57s
1

208:	learn: 0.1549851	total: 684ms	remaining: 2.59s
209:	learn: 0.1547133	total: 688ms	remaining: 2.59s
210:	learn: 0.1543980	total: 691ms	remaining: 2.58s
211:	learn: 0.1541448	total: 695ms	remaining: 2.58s
212:	learn: 0.1538665	total: 698ms	remaining: 2.58s
213:	learn: 0.1535546	total: 702ms	remaining: 2.58s
214:	learn: 0.1533492	total: 705ms	remaining: 2.57s
215:	learn: 0.1530063	total: 709ms	remaining: 2.57s
216:	learn: 0.1525816	total: 712ms	remaining: 2.57s
217:	learn: 0.1522022	total: 715ms	remaining: 2.56s
218:	learn: 0.1519405	total: 718ms	remaining: 2.56s
219:	learn: 0.1515664	total: 721ms	remaining: 2.56s
220:	learn: 0.1512910	total: 725ms	remaining: 2.55s
221:	learn: 0.1510844	total: 728ms	remaining: 2.55s
222:	learn: 0.1507056	total: 732ms	remaining: 2.55s
223:	learn: 0.1503881	total: 735ms	remaining: 2.54s
224:	learn: 0.1500933	total: 738ms	remaining: 2.54s
225:	learn: 0.1498782	total: 741ms	remaining: 2.54s
226:	learn: 0.1496065	total: 745ms	remaining: 2.54s
227:	learn: 

367:	learn: 0.1237832	total: 1.19s	remaining: 2.04s
368:	learn: 0.1235694	total: 1.19s	remaining: 2.04s
369:	learn: 0.1233468	total: 1.2s	remaining: 2.04s
370:	learn: 0.1232406	total: 1.2s	remaining: 2.03s
371:	learn: 0.1231755	total: 1.2s	remaining: 2.03s
372:	learn: 0.1230266	total: 1.21s	remaining: 2.03s
373:	learn: 0.1227788	total: 1.21s	remaining: 2.02s
374:	learn: 0.1226153	total: 1.21s	remaining: 2.02s
375:	learn: 0.1223892	total: 1.22s	remaining: 2.02s
376:	learn: 0.1222406	total: 1.22s	remaining: 2.02s
377:	learn: 0.1221314	total: 1.22s	remaining: 2.01s
378:	learn: 0.1219972	total: 1.23s	remaining: 2.01s
379:	learn: 0.1218164	total: 1.23s	remaining: 2s
380:	learn: 0.1216690	total: 1.23s	remaining: 2s
381:	learn: 0.1214373	total: 1.23s	remaining: 2s
382:	learn: 0.1212976	total: 1.24s	remaining: 1.99s
383:	learn: 0.1211284	total: 1.24s	remaining: 1.99s
384:	learn: 0.1208891	total: 1.24s	remaining: 1.99s
385:	learn: 0.1207728	total: 1.25s	remaining: 1.99s
386:	learn: 0.1206498	to

577:	learn: 0.0992538	total: 1.87s	remaining: 1.36s
578:	learn: 0.0991994	total: 1.87s	remaining: 1.36s
579:	learn: 0.0990546	total: 1.88s	remaining: 1.36s
580:	learn: 0.0989605	total: 1.88s	remaining: 1.35s
581:	learn: 0.0989317	total: 1.88s	remaining: 1.35s
582:	learn: 0.0987782	total: 1.89s	remaining: 1.35s
583:	learn: 0.0987449	total: 1.89s	remaining: 1.34s
584:	learn: 0.0986371	total: 1.89s	remaining: 1.34s
585:	learn: 0.0985391	total: 1.9s	remaining: 1.34s
586:	learn: 0.0985062	total: 1.9s	remaining: 1.34s
587:	learn: 0.0983881	total: 1.9s	remaining: 1.33s
588:	learn: 0.0983735	total: 1.91s	remaining: 1.33s
589:	learn: 0.0983361	total: 1.91s	remaining: 1.33s
590:	learn: 0.0983124	total: 1.91s	remaining: 1.32s
591:	learn: 0.0981533	total: 1.92s	remaining: 1.32s
592:	learn: 0.0980416	total: 1.92s	remaining: 1.32s
593:	learn: 0.0980005	total: 1.92s	remaining: 1.31s
594:	learn: 0.0979901	total: 1.93s	remaining: 1.31s
595:	learn: 0.0978734	total: 1.93s	remaining: 1.31s
596:	learn: 0.0

781:	learn: 0.0809956	total: 2.54s	remaining: 710ms
782:	learn: 0.0808643	total: 2.55s	remaining: 706ms
783:	learn: 0.0807960	total: 2.55s	remaining: 703ms
784:	learn: 0.0806905	total: 2.56s	remaining: 700ms
785:	learn: 0.0806150	total: 2.56s	remaining: 697ms
786:	learn: 0.0804574	total: 2.56s	remaining: 693ms
787:	learn: 0.0803377	total: 2.56s	remaining: 690ms
788:	learn: 0.0802692	total: 2.57s	remaining: 687ms
789:	learn: 0.0802137	total: 2.57s	remaining: 684ms
790:	learn: 0.0801627	total: 2.57s	remaining: 680ms
791:	learn: 0.0800801	total: 2.58s	remaining: 677ms
792:	learn: 0.0800750	total: 2.58s	remaining: 674ms
793:	learn: 0.0799865	total: 2.58s	remaining: 670ms
794:	learn: 0.0798883	total: 2.59s	remaining: 667ms
795:	learn: 0.0797720	total: 2.59s	remaining: 664ms
796:	learn: 0.0797013	total: 2.59s	remaining: 661ms
797:	learn: 0.0795693	total: 2.6s	remaining: 657ms
798:	learn: 0.0795483	total: 2.6s	remaining: 654ms
799:	learn: 0.0794433	total: 2.6s	remaining: 651ms
800:	learn: 0.0

986:	learn: 0.0647995	total: 3.22s	remaining: 42.5ms
987:	learn: 0.0647772	total: 3.23s	remaining: 39.2ms
988:	learn: 0.0646999	total: 3.23s	remaining: 35.9ms
989:	learn: 0.0646313	total: 3.23s	remaining: 32.7ms
990:	learn: 0.0645942	total: 3.24s	remaining: 29.4ms
991:	learn: 0.0645220	total: 3.24s	remaining: 26.1ms
992:	learn: 0.0644469	total: 3.24s	remaining: 22.9ms
993:	learn: 0.0643889	total: 3.25s	remaining: 19.6ms
994:	learn: 0.0643001	total: 3.25s	remaining: 16.3ms
995:	learn: 0.0642019	total: 3.25s	remaining: 13.1ms
996:	learn: 0.0640832	total: 3.26s	remaining: 9.8ms
997:	learn: 0.0640485	total: 3.26s	remaining: 6.53ms
998:	learn: 0.0639593	total: 3.26s	remaining: 3.27ms
999:	learn: 0.0638859	total: 3.27s	remaining: 0us


array([[940,  10],
       [ 39, 122]], dtype=int64)

Accuracy 0.9558955895589559
Recall 0.7577639751552795
Precision 0.9242424242424242
F1 0.8327645051194539


In [32]:
# How to check if the model is overfitted
recall_score(y_train, [eval(i) for i in model1.predict(X_train)])

0.8913043478260869